# AI - Vacuum agent, empty program

Created (R. Basili, Nov. 2020)  
Modified (C.D. Hromei, Oct 2024)

## Exercises

1. Fill the program of the `Agent` in order to reproduce the behaviour of random moves.
2. Fill the program of the `Agent` in order to search for the closest dirty room, go there and `Suck`.  
    **Hint**: you should modify the `Perception` method of the Environment as well to make it Completely Observable.
3. Modify the `Environment` locations into a 2x2 matrix instead of a row.
4. Modify the `Execute action` method of the Environment in order to implement a probabilistic failure of the action chosen by the Agent. In that case, the action should fail and nothing happens!

In [1]:
from tkinter import *
import random
from agents import *
from PIL import Image

collections.Callable = collections.abc.Callable

loc_A, loc_B, loc_C = (0, 0), (1, 0), (2,0)  # The three locations for the Vacuum world

class Gui(Environment):

    """This GUI environment has three locations, A, B and C. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status."""

    def __init__(self, root, height=400, width=520):
        super().__init__()
        self.status = {loc_A: 'Dirty',
                       loc_B: 'Dirty', 
                       loc_C: 'Clean'}
        self.root = root
        self.height = height
        self.width = width
        self.canvas = None
        self.buttons = []
        self.create_canvas()
        self.create_buttons()

    def thing_classes(self):
        """The list of things which can be used in the environment."""
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        print('Perceived Location=', agent.location,' - Perceived StatusAtLoc=',  self.status[agent.location])
        
        return (agent.location, self.status[agent.location]) #(agent.location, self.status[agent.location])

    def execute_action(self, agent, action):
        """Change the location status (Dirty/Clean); track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        print('Executing action: ', action)
        if action == 'Right':
            if agent.location == loc_A:
                agent.location=loc_B
            else:
                agent.location=loc_C
            agent.performance -= 1
        elif action == 'Left':
            if agent.location == loc_C:
                agent.location = loc_B
            else:
                agent.location=loc_A
            agent.performance -= 1
        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                if agent.location == loc_A:
                    self.buttons[0].config(bg='white', activebackground='light grey')
                elif agent.location == loc_B:
                    self.buttons[1].config(bg='white', activebackground='light grey')
                else:
                    self.buttons[2].config(bg='white', activebackground='light grey')
                agent.performance += 10
            self.status[agent.location] = 'Clean'

    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice([loc_A, loc_B, loc_C])

    ###################################################################################################
    #                        INTERFACE METHODS NOT USEFULL FOR THE AGENT/ENVIRONMENT
    ###################################################################################################
    def create_canvas(self):
        """Creates Canvas element in the GUI."""
        self.canvas = Canvas(
            self.root,
            width=self.width,
            height=self.height,
            background='powder blue')
        self.canvas.pack(side='bottom')

    def create_buttons(self):
        """Creates the buttons required in the GUI."""
        button_left = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_left.config(command=lambda btn=button_left: self.dirt_switch(btn))
        self.buttons.append(button_left)
        button_left_window = self.canvas.create_window(130, 200, anchor=N, window=button_left)

        button_middle = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_middle.config(command=lambda btn=button_middle: self.dirt_switch(btn))
        self.buttons.append(button_middle)
        button_middle_window = self.canvas.create_window(250, 200, anchor=N, window=button_middle)

        button_right = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_right.config(command=lambda btn=button_right: self.dirt_switch(btn))
        self.buttons.append(button_right)
        button_right_window = self.canvas.create_window(370, 200, anchor=N, window=button_right)

    def dirt_switch(self, button):
        """Gives user the option to put dirt in any tile."""
        bg_color = button['bg']
        if bg_color == 'saddle brown':
            button.config(bg='white', activebackground='light grey')
        elif bg_color == 'white':
            button.config(bg='saddle brown', activebackground='light goldenrod')

    def read_env(self):
        """Reads the current state of the GUI."""
        for i, btn in enumerate(self.buttons):
            if i == 0:
                if btn['bg'] == 'white':
                    self.status[loc_A] = 'Clean'
                else:
                    self.status[loc_A] = 'Dirty'
            elif i == 1:
                if btn['bg'] == 'white':
                    self.status[loc_B] = 'Clean'
                else:
                    self.status[loc_B] = 'Dirty'
            else:
                if btn['bg'] == 'white':
                    self.status[loc_C] = 'Clean'
                else:
                    self.status[loc_C] = 'Dirty'

    def update_env(self, agent):
        """Updates the GUI according to the agent's action."""
        self.read_env()
        # print(self.status)
        before_step = agent.location
        self.step()   #execute_actions
        print('Current Environment Status: ', self.status)
        print('Current Agent Location: ',agent.location)
        move_agent(self, agent, before_step)

###################################################################################################
#                                           THE AGENT
###################################################################################################

def ReflexVacuumAgent():
    """A reflex agent for the two-state vacuum environment. [Figure 2.8]
    >>> agent = ReflexVacuumAgent()
    >>> environment = TrivialVacuumEnvironment()
    >>> environment.add_thing(agent)
    >>> environment.run()
    >>> environment.status == {(1,0):'Clean' , (0,0) : 'Clean'}
    True
    """
    
    def program(percept):
        
        location, status = percept
        print('Perceived Status: ', percept)
        
        action = random.choice(["Left", "Right"])
        
        if status == "Dirty":
            action = "Suck"
        elif status == "Clean":
            if location == loc_A:
                action = "Right"
            elif location == loc_C:
                action = "Left"
        
        return action    
        
    return Agent(program)

###################################################################################################
#                            INTERFACE METHODS TO CREATE AND MOVE THE AGENT
###################################################################################################

def create_agent(env, agent, MyImage):
    """Creates the agent in the GUI and is kept independent of the environment."""
    env.add_thing(agent)

    if agent.location == (0, 0):
        env.text = env.canvas.create_text(120, 140, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(80, 100, image=MyImage)
        
    elif agent.location == (2, 0):
        env.text = env.canvas.create_text(360, 140, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(320, 100, image=MyImage)
    elif agent.location == (1, 0):
        env.text = env.canvas.create_text(240, 140, font="Helvetica 10 bold italic", text="Agent")
        env.agent_logo = env.canvas.create_image(200, 100, image=MyImage)

def move_agent(env, agent, before_step):
    """Moves the agent in the GUI when 'next' button is pressed."""
    if agent.location == before_step:
        pass
    else:
        if agent.location == (2, 0) and before_step == (1,0):
            env.canvas.move(env.text, 120, 0)
            env.canvas.move(env.agent_logo, 120, 0)
        elif agent.location == (1, 0) and  before_step == (2,0): 
            env.canvas.move(env.text, -120, 0)
            env.canvas.move(env.agent_logo, -120, 0)
        elif agent.location == (1, 0) and  before_step == (0,0): 
            env.canvas.move(env.text, 120, 0)
            env.canvas.move(env.agent_logo, 120, 0)
        elif agent.location == (0, 0) and before_step == (1,0):
            env.canvas.move(env.text, -120, 0)
            env.canvas.move(env.agent_logo, -120, 0)

# TODO: Add more agents to the environment.
# TODO: Expand the environment to XYEnvironment.

###################################################################################################
#                                           MAIN LOOP
###################################################################################################
def main():
    """The main function of the program."""
    root = Tk()
    root.title("Vacuum Environment")
    root.geometry("500x500")
    root.resizable(0, 0)
    frame = Frame(root, bg='black')
    
    path = "AgentVacuum.png"
    
    try:
        MyImage = PhotoImage(file=path)  # Using PhotoImage for .png
    except Exception as e:
        print(f"Error loading image: {e}")
        return

    next_button = Button(frame, text='Next', height=2, width=6, padx=2, pady=2)
    next_button.pack(side='left')
    frame.pack(side='bottom')
    
    env = Gui(root)
    agent = ReflexVacuumAgent()
    create_agent(env, agent, MyImage)
    
    next_button.config(command=lambda: env.update_env(agent))
    
    root.mainloop()


if __name__ == "__main__":
    main()


Perceived Location= (1, 0)  - Perceived StatusAtLoc= Clean
Perceived Status:  ((1, 0), 'Clean')
Executing action:  Left
Current Environment Status:  {(0, 0): 'Dirty', (1, 0): 'Clean', (2, 0): 'Clean'}
Current Agent Location:  (0, 0)
Perceived Location= (0, 0)  - Perceived StatusAtLoc= Dirty
Perceived Status:  ((0, 0), 'Dirty')
Executing action:  Suck
Current Environment Status:  {(0, 0): 'Clean', (1, 0): 'Clean', (2, 0): 'Clean'}
Current Agent Location:  (0, 0)
Perceived Location= (0, 0)  - Perceived StatusAtLoc= Clean
Perceived Status:  ((0, 0), 'Clean')
Executing action:  Right
Current Environment Status:  {(0, 0): 'Clean', (1, 0): 'Clean', (2, 0): 'Clean'}
Current Agent Location:  (1, 0)
Perceived Location= (1, 0)  - Perceived StatusAtLoc= Clean
Perceived Status:  ((1, 0), 'Clean')
Executing action:  Right
Current Environment Status:  {(0, 0): 'Clean', (1, 0): 'Clean', (2, 0): 'Clean'}
Current Agent Location:  (2, 0)
Perceived Location= (2, 0)  - Perceived StatusAtLoc= Clean
Perceiv